Concept notebook to explore colorbar implementation in bqplot vs Matplotlib.

Matplotlib example is from https://docs.astropy.org/en/latest/visualization/normalization.html .

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Generate a test image
image = np.arange(65536).reshape((256, 256))

In [ ]:
from astropy.visualization import simple_norm

# Create an ImageNormalize object
norm = simple_norm(image, 'sqrt')

# Display the image
fig, ax = plt.subplots()
im = ax.imshow(image, origin='lower', norm=norm)
fig.colorbar(im, location="bottom")

How, how does Imviz fare?

In [ ]:
from jdaviz import Imviz

In [ ]:
imviz = Imviz()
imviz.load_data(image, data_label="test image")
imviz.show()

In [ ]:
plg = imviz.plugins["Plot Options"]

In [ ]:
plg.open_in_tray()

In [ ]:
# Colormap (to compare with Matplotlib above)
plg.image_colormap = "Viridis"

In [ ]:
# Monochromatic
plg.image_color_mode = "Monochromatic"
plg.image_color = "red"

In [ ]:
plg.stretch_function = "Square Root"

In [ ]:
imviz.default_viewer.zoom_level = "fit"

In [ ]:
from astropy.visualization import ManualInterval, ContrastBiasStretch
from glue.config import colormaps, stretches
from glue.viewers.image.composite_array import COLOR_CONVERTER

# Copied from glue/viewers/image/composite_array.py

color_mode = plg._obj.image_color_mode_value
interval = ManualInterval(vmin=plg._obj.stretch_vmin_value, vmax=plg._obj.stretch_vmax_value)
contrast_bias = ContrastBiasStretch(plg._obj.image_contrast_value, plg._obj.image_bias_value)
stretch = stretches.members[plg._obj.stretch_function_value]
array = plg._obj.stretch_histogram.figure.marks[0].x

data = interval(array)
data = contrast_bias(data, out=data)
data = stretch(data, out=data)

if color_mode == 'Colormaps':
    cmap = colormaps[plg._obj.image_colormap.text]
    if hasattr(cmap, "get_bad"):
        bad_color = cmap.get_bad().tolist()[:3]
        layer_cmap = cmap.with_extremes(bad=bad_color + [plg._obj.image_opacity_value])
    else:
        layer_cmap = cmap

    # Compute colormapped image
    plane = layer_cmap(data)

else:  # Monochromatic
    # Get color
    color = COLOR_CONVERTER.to_rgba_array(plg._obj.image_color_value)[0]
    plane = data[:, np.newaxis] * color
    plane[:, 3] = 1

plane = np.clip(plane, 0, 1, out=plane)

In [ ]:
import matplotlib

ipycolors = [matplotlib.colors.rgb2hex(p, keep_alpha=False) for p in plane]

In [ ]:
# This draws dots across the histogram. Drawing line does not work. ynorm=True does not put it on top.
#x = plg._obj.stretch_histogram.figure.marks[0].x
#plg._obj.stretch_histogram.add_scatter('colorbar', x=x, y=np.ones(x.size), ynorm=True, color=ipycolors)

In [ ]:
# This encodes colorbar inside histogram bars
plg._obj.stretch_histogram.figure.marks[1].colors = ipycolors

In [ ]:
# Re-displays the histogram
plg._obj.stretch_histogram

In [ ]:
# Matplotlib rendering of the same colors, as reassurance
x = plg._obj.stretch_histogram.figure.marks[0].x
plt.scatter(x, np.ones(x.size), color=ipycolors)